In [69]:
import numpy as np

In [70]:
import pandas as pd

In [71]:
import matplotlib.pyplot as plt

# Assignment

- The goal is to write functions that performs elementary row operations on a given matrix, we would write a function to add the rows of that matrix, swap the rows of a matrix, and finally multiply a row by a scalar.

<a name='1'></a>
## 1 - System of Linear Equations and Corresponding `NumPy` Arrays

Matrices can be used to solve systems of equations. But first, you need to represent the system using matrices. Given the following system of linear equations:

$$\begin{cases} 
2x_1-x_2+x_3+x_4=6, \\ x_1+2x_2-x_3-x_4=3, \\ -x_1+2x_2+2x_3+2x_4=14, \\ x_1-x_2+2x_3+x_4=8, \end{cases}\tag{1}$$

you will construct matrix $A$, where each row represents one equation in the system and each column represents a variable $x_1$, $x_2$, $x_3$, $x_4$. The free coefficients from the right sides of the equations you will put into vector $b$.

In [72]:
A = np.array([     
        [2, -1, 1, 1],
        [1, 2, -1, -1],
        [-1, 2, 2, 2],
        [1, -1, 2, 1]    
    ], dtype=np.dtype(float)) 
b = np.array([6, 3, 14, 8], dtype=np.dtype(float))

In [73]:
def MultiplyRow(M: np.ndarray, row_index: int, row_multiple: float) -> np.ndarray:
    # we are going to copy the matrix so as to avoid editing the main matrix passed into it
    M_copy = M.copy()
    M_copy[row_index] = row_multiple * M_copy[row_index]
    return M_copy

In [74]:
def AddRows(M: np.ndarray, row_1_index: int, row_2_index: int, row_num_1_multiple: float) -> np.ndarray:
    M_copy = M.copy()
    M_copy[row_2_index] = row_num_1_multiple * M_copy[row_1_index] + M_copy[row_2_index]
    return M_copy

In [75]:
def SwapRows(M: np.ndarray, row_1_index: int, row_2_index: int) -> np.ndarray:
    M_copy = M.copy()
    temp = M_copy[row_1_index].copy()
    M_copy[row_1_index] = M_copy[row_2_index]
    M_copy[row_2_index] = temp

    return M_copy

In [76]:
def ScaleRow(M: np.ndarray, row_index: int, pivot_element: float) -> None:
    M_new = M.copy()
    M_new[row_index] = M_new[row_index] / pivot_element
    return M_new[row_index]

    

In [77]:
print('Swapping rows')
print(SwapRows(A, 2, 3))

Swapping rows
[[ 2. -1.  1.  1.]
 [ 1.  2. -1. -1.]
 [ 1. -1.  2.  1.]
 [-1.  2.  2.  2.]]


In [78]:
# my goal is now to use these new functions I have defined to perform elementary row operations on the given matrix, and
# transform the matrix into reduced echelon form.

In [79]:

# first we want to create an augmented matrix by joining the coefficient of the matrix A with their respective constants

In [80]:
augmented_matrix = np.hstack((A, b.reshape((4,1))))
augmented_matrix

array([[ 2., -1.,  1.,  1.,  6.],
       [ 1.,  2., -1., -1.,  3.],
       [-1.,  2.,  2.,  2., 14.],
       [ 1., -1.,  2.,  1.,  8.]])

In [81]:
# next I am going to define a function that performs the elementary row operations on the matrix until it is reduced to its echelon form

In [114]:
def matrix_to_echelon(M: np.ndarray) -> np.ndarray:
    M_new: np.ndarray = M.copy()

    # what I would like to do is divide the first row by its first element, so that I can get a one as my pivot
    pivot: int = M_new[0][0]
    
    M_new[0] = ScaleRow(M_new, 0, pivot)

    # Eliminating the pivot in row 2 under the pivot in row 1, we would do this for the other rows of the matrix
    M_new = AddRows(M_new, 0, 1, -1)

    M_new = AddRows(M_new, 0, 2, 1)

    M_new = AddRows(M_new, 0, 3, -1)

    # Now, we would eliminate all the pivots in row 3 under the pivot in row 2, but lets scale row2 so that we have a 1 as the pivot

    M_new[1] = ScaleRow(M_new, 1, M_new[1][1])

    # continuing with addition operation

    M_new = AddRows(M_new, 1, 2, -1.5)

    M_new = AddRows(M_new, 1, 3, 0.5)

    # We scale r3 by its pivot

    M_new[2] = ScaleRow(M_new, 2, M_new[2][2])

    # continuing with the addition operation

    M_new = AddRows(M_new, 2, 3, -1.2)

    # we can leave it at this because the matrix is already in echelon form, but we want 1 as the pivot so we scale the fourth row by its pivot

    M_new[3] = ScaleRow(M_new, 3, M_new[3][3])

    return M_new

A_ref = matrix_to_echelon(augmented_matrix)

In [116]:
A_ref

array([[ 1. , -0.5,  0.5,  0.5,  3. ],
       [ 0. ,  1. , -0.6, -0.6,  0. ],
       [ 0. ,  0. ,  1. ,  1. ,  5. ],
       [-0. , -0. , -0. ,  1. ,  1. ]])

<a name='ex05'></a>
### Exercise 5

From the last line of the reduced matrix `A_ref` find $x_4$. Then you can calculate each of the $x_3$, $x_2$ and $x_1$ taking the elements of the matrix `A_ref[i,j]` and solving the linear equations one by one.

In [123]:
x4 = 1
x3 = A_ref[2,4] - A_ref[2,3] * x4
x2 = A_ref[1,4] - A_ref[1,3] * x4 - A_ref[1,2] * x3
x1 = A_ref[0,4] - A_ref[0,3] * x4 - A_ref[0,2] * x3 - A_ref[0,1] * x2

print(x1, x2, x3, x4)

2.0 3.0 4.0 1


<a name='ex06'></a>
### Exercise 6

Using the same elementary operations as above you can reduce the matrix further to diagonal form, from which you can see the solutions easily.

In [138]:
def echelon_to_reduced_echelon(M: np.ndarray) -> np.ndarray:
    M_new = M.copy()

    M_new = AddRows(M_new, 3, 0, -0.5)
    M_new = AddRows(M_new, 3, 1, 0.6)
    M_new = AddRows(M_new, 3, 2, -1)
    M_new = AddRows(M_new, 2, 0, -0.5)
    M_new = AddRows(M_new, 2, 1, 0.6)
    M_new = AddRows(M_new, 1, 0, 0.5)

    return M_new

In [139]:
reduced_echelon = echelon_to_reduced_echelon(A_ref)
reduced_echelon

array([[ 1.,  0.,  0.,  0.,  2.],
       [ 0.,  1.,  0.,  0.,  3.],
       [ 0.,  0.,  1.,  0.,  4.],
       [-0., -0., -0.,  1.,  1.]])

- From this, we can deduce that the matrix has a rank of 4, and it is linearly independent, and it is non-singular

In [140]:
# when performing row operations on matrices, you have to be careful, so as not to run into errors in your program